In [1]:
require 'nn'
require 'hdf5'

### LOADING THE DATA AND CONVERTING IT TO LOGREG FW:

In [2]:
myFile = hdf5.open('PTB.hdf5','r')

In [3]:
data = myFile:all()
myFile:close()

In [35]:
data

{
  train_output : LongTensor - size: 912666
  train_input_cap_windows : LongTensor - size: 912666x5
  test_input_word_windows : LongTensor - size: 129696x5
  valid_output : LongTensor - size: 131808
  valid_input_cap_windows : LongTensor - size: 131808x5
  nwords : IntTensor - size: 1


  test_input_cap_windows : LongTensor - size: 129696x5
  train_input_word_windows : LongTensor - size: 912666x5
  nclasses : IntTensor - size: 1
  word_embeddings : DoubleTensor - size: 400002x50
  valid_input_word_windows : LongTensor - size: 131808x5
}


In [4]:
train_input_word_windows = data['train_input_word_windows']
train_output = data['train_output']

In [5]:
train = train_input_word_windows:clone()

In [6]:
train_input_cap_windows = data['train_input_cap_windows']
train_cap = train_input_cap_windows:clone()
for j = 1, 5 do
    train:narrow(2,j,1):add((j-1)*100002)
end
for j = 1, 5 do
    train_cap:narrow(2,j,1):add((j-1)*4)
end

### TRAINING:

In [84]:
linreg = nn.Sequential()

In [85]:
par = nn.ParallelTable()

In [86]:
par:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1])) -- first child
par:add(nn.LookupTable(5*4,data['nclasses'][1])) 

In [87]:
linreg_wc = nn.Sequential()
linreg_wc:add(par)
linreg_wc:add(nn.CAddTable())
linreg_wc:add(nn.Sum(2))
linreg_wc:add(nn.Add(45))
linreg_wc:add(nn.LogSoftMax())

In [14]:
-- Sanity check:
linreg_wc:forward({train:narrow(1,5, 10),train_cap:narrow(1,5, 10)})

Columns 1 to 8
 -9.4556  -9.0840  -9.1315  -6.5516  -6.7180  -9.7721  -5.2577  -4.9102
 -5.2287  -2.2692  -8.3143  -9.3451  -7.0764  -4.7653  -9.5746  -9.7060
 -1.9097 -10.2818  -9.5193  -8.5484  -6.1988 -11.7294  -4.8129  -6.0074
 -6.0959  -6.8740  -7.0861  -3.9074  -3.3044 -13.0307  -6.0800  -1.8501
 -6.6274  -6.9709 -10.3367 -10.9805  -3.8520  -9.9096  -9.7327  -9.6333
 -3.9007 -10.0580  -4.2999  -5.2826  -1.2004  -6.2150  -8.8340  -7.0047
 -7.2727  -9.6507  -7.1163  -6.6166  -6.0662 -11.5380  -5.1602  -5.9396
 -6.2306  -9.3752  -7.4906  -7.3687  -9.2647  -8.5895  -6.3623  -6.4497
 -9.5701  -7.8016  -5.1224 -11.1787  -4.3156 -10.3472  -7.7270  -8.0237
 -7.2037 -10.6715 -10.0518  -7.5502  -0.4954 -11.1519  -6.4436  -6.7654

Columns 9 to 16
-11.5083  -9.1985  -1.7827  -7.7293 -10.0467  -7.8615  -7.8726 -11.5432
 -9.3059  -6.2154  -7.4380 -11.2678  -6.1378 -10.6858  -8.7199  -7.6150
 -7.3944  -6.5481  -5.6413  -8.7201  -9.8178  -7.4246  -9.8617  -6.8240
 -7.7495  -2.9847  -5.8901  -7.5

977  -1.8450
-12.2412  -9.7217  -8.9921  -9.7677  -1.3894
[torch.DoubleTensor of size 10x45]



In [ ]:
-- linreg_w:add(nn.LookupTable(5*data['nwords'][1],data['nclasses'][1]))
-- linreg_w:add(nn.Sum(2))
-- linreg_w:add(nn.Add(data['nclasses'][1]))
-- linreg_w:add(nn.LogSoftMax())

In [88]:
criterion = nn.ClassNLLCriterion()

In [ ]:
eta = 0.1

max_e = 15

batchsize = 32

input_w = torch.Tensor(batchsize,5)
input_c = torch.Tensor(batchsize,5)
output = torch.Tensor(batchsize)

loss_tensor = torch.Tensor(90)
k = 1

for i = 1,max_e do
    print("EPOCH: "..i)
    
    for j = 1,torch.floor(train:size(1)/batchsize) do
        linreg_wc:zeroGradParameters()
        
        input_w = train:narrow(1, (j-1)*batchsize+1, batchsize)
        input_c = train_cap:narrow(1, (j-1)*batchsize+1, batchsize)
        preds = linreg_wc:forward({input_w,input_c})
        
        output = train_output:narrow(1,(j-1)*batchsize+1, batchsize)
        
        loss = criterion:forward(preds, output)
        
        if j % 500 == 0 then
            loss_tensor[k] = loss
            k = k + 1
        end
        
        dLdpreds = criterion:backward(preds, output)
        
        linreg_wc:backward({input_w,input_c}, dLdpreds)
        
        linreg_wc:updateParameters(eta)
        
    end
    
end


EPOCH: 1	


### TRAINING ACCU: 

In [13]:
preds_train = linreg_wc:forward({train,train_cap})

In [30]:
train_output[1]

1	


In [32]:
a[1][1]

1	


In [15]:
m,a = preds_train:max(2)

In [34]:
acc = 0
for i = 1, train:size(1) do
    if a[i][1] == train_output[i] then
        acc = acc + 1
    end
end
print("Accuracy on train is: "..acc/train:size(1))

Accuracy on train is: 0.49810883718688	


### VALIDATION:

In [36]:
val_word = data['valid_input_word_windows']:clone()
val_cap = data['valid_input_cap_windows']:clone()

for j = 1, 5 do
    val_word:narrow(2,j,1):add((j-1)*100002)
end

for j = 1, 5 do
    val_cap:narrow(2,j,1):add((j-1)*4)
end

In [39]:
pred_val = linreg_wc:forward({val_word,val_cap})

In [40]:
m_v,a_v = pred_val:max(2)

In [42]:
acc_v = 0
for i = 1, data['valid_output']:size(1) do
    if a_v[i][1] == data['valid_output'][i] then
        acc_v = acc_v + 1
    end
end
print("Accuracy on validation is: "..acc_v/val_word:size(1))

Accuracy on validation is: 0.50381615683418	


## Neural Network:

In [49]:
hp1 = 50

In [78]:
nn1 = nn.Sequential()

In [79]:
par2 = nn.ParallelTable()
par2:add(nn.LookupTable(data['nwords'][1],hp1)) -- first child
par2:add(nn.LookupTable(4,hp1)) 

In [80]:
nn1:add(par2)
nn1:add(nn.CAddTable())
nn1:add(nn.Sum(2))
nn1:add(nn.Add(hp1))
nn1:add(nn.HardTanh())
nn1:add(nn.Linear(hp1,45))

In [75]:
criterion2 = nn.ClassNLLCriterion()

In [76]:
timer = torch.Timer()

eta = 0.01
max_e = 1
hp1 = 50
batchsize = 100
input_w = torch.Tensor(100,5)
input_c = torch.Tensor(100,5)
output = torch.Tensor(100)

loss_tensor = torch.Tensor(18*max_e)

k = 1

for i = 1,max_e do
    print("EPOCH: "..i)
    
    for j = 1,torch.floor(train:size(1)/batchsize) do
        nn1:zeroGradParameters()
        
        input_w = train_input_word_windows:narrow(1, (j-1)*batchsize+1, batchsize)
        input_c = train_input_cap_windows:narrow(1, (j-1)*batchsize+1, batchsize)
        preds = nn1:forward({input_w,input_c})
        
        output = train_output:narrow(1,(j-1)*batchsize+1, batchsize)
        
        loss = criterion2:forward(preds, output)
        
        if j % 500 == 0 then
            loss_tensor[k] = loss
            k = k + 1
        end
        
        dLdpreds = criterion2:backward(preds, output)
        
        nn1:backward({input_w,input_c}, dLdpreds)
        
        nn1:updateParameters(eta)
        
    end
    
end

print('Time elapsed for 1 epoch: ' .. timer:time().real .. ' seconds')

EPOCH: 1	


Time elapsed for 1 epoch: 104.36425614357 seconds	


In [77]:
loss_tensor

 -10.4673
 -20.1917
 -40.5038
 -55.5550
 -67.3850
 -94.7285
-116.4675
-132.2956
-146.8417
-142.9090
-188.5284
-211.0211
-257.5870
-231.1713
-217.2197
-257.8606
-309.2840
-330.2755
[torch.DoubleTensor of size 18]

